In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer

from hashlib import sha256
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf

pd.set_option('display.max_columns', None)

output = join("output","correcoes","tb_pacientes")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer.reset()
tb = TbPacientes()
tb.update()
timer.stop()

109.67933773994446

In [3]:
timer.reset()
casos_confirmados = CasosConfirmados()
# casos_confirmados.update()
timer.stop()

1.192655324935913

In [4]:
casost = tb.get()
casosc = casos_confirmados.get_casos()
print(f"casos casos_confirmados: {casosc.shape}")
print(f"casos tb_pacientes: {casost.shape}")

casos casos_confirmados: (399065, 21)
casos tb_pacientes: (399065, 48)


In [5]:
idx_casos_confirmados = casosc[casosc['hash'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_tb_pacientes = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')

In [6]:
idx_tb_pacientes = casost[casost['hash'].isin(casosc['hash'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_less'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_more'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_less_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_more_atend'])].index.tolist()
idx_tb_pacientes = set(idx_tb_pacientes)
tb_pacientes_nao_casos_confirmados = casost.loc[ set(casost.index.tolist()) - idx_tb_pacientes ].sort_values('identificacao')


In [7]:
print(f"casos_confirmados_nao_tb_pacientes: {casos_confirmados_nao_tb_pacientes.shape}")
if len(casos_confirmados_nao_tb_pacientes) > 0:
    casos_confirmados_nao_tb_pacientes.sort_values('nome').to_excel(join(output,'casos_confirmados_nao_tb_pacientes.xlsx'), index=False)
print(f"tb_pacientes_nao_casos_confirmados: {tb_pacientes_nao_casos_confirmados.shape}")
if len(tb_pacientes_nao_casos_confirmados) > 0:
    tb_pacientes_nao_casos_confirmados.sort_values('nome').to_excel(join(output,'tb_pacientes_nao_casos_confirmados.xlsx'), index=False)

casos_confirmados_nao_tb_pacientes: (0, 21)
tb_pacientes_nao_casos_confirmados: (0, 48)


In [8]:
casosc = casosc.sort_values('nome').drop(columns=['hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend'])
casost = casost.sort_values('nome').drop(columns=['hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend'])

In [9]:
casost['ibge_res_pr'] = casost['ibge_res_pr'].fillna('9999999')
casost['ibge_atend_pr'] = casost['ibge_atend_pr'].fillna('9999999')
casosc['ibge7'] = casosc['ibge7'].fillna('9999999')

In [10]:
writer = pd.ExcelWriter(join(output,"novos_tb_pacientes.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casost.to_excel(writer,index=None)

writer.save()
writer.close()

In [11]:
writer = pd.ExcelWriter(join(output,"casos_ordenados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casosc.to_excel(writer,index=None)

writer.save()
writer.close()